## 불균형 데이터 처리

### 라이브러리 준비

In [2]:
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 8.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### 데이터 준비

In [2]:
import requests
import io

# 이 데이터는 신용카드 사기 거래 탐지를 위한 데이터셋입니다.
# 'creditcard.csv' 파일에는 신용카드 거래 정보와 사기 여부가 포함되어 있습니다.
url = "https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv"
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text))

print("데이터 shape:", data.shape)
print("\n데이터 샘플:")
print(data.head())
print("\n클래스 분포:")
print(data['Class'].value_counts(normalize=True))


데이터 shape: (284807, 31)

데이터 샘플:
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   


### 데이터 분리

In [3]:
### 레이블 분리
X = data.drop('Class', axis=1)
y = data['Class']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### SMOTE 적용

In [4]:
# SMOTE 적용
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

### 분포 시각화

In [5]:
# 원본 데이터와 SMOTE 적용 후 데이터의 클래스 분포 계산
original_distribution = y_train.value_counts().sort_index()
resampled_distribution = y_train_resampled.value_counts().sort_index()

# 서브플롯 생성
fig = make_subplots(rows=1, cols=2, subplot_titles=('원본 데이터 분포', 'SMOTE 적용 후 데이터 분포'))

# 원본 데이터 분포 그래프
fig.add_trace(
    go.Bar(x=['정상', '사기'], y=original_distribution.values, name='원본 데이터'),
    row=1, col=1
)

# SMOTE 적용 후 데이터 분포 그래프
fig.add_trace(
    go.Bar(x=['정상', '사기'], y=resampled_distribution.values, name='SMOTE 적용 후'),
    row=1, col=2
)

# 레이아웃 설정
fig.update_layout(
    title='불균형 데이터셋 분포 비교',
    height=500,
    width=1000
)

# y축 레이블 설정
fig.update_yaxes(title_text='데이터 수', row=1, col=1)
fig.update_yaxes(title_text='데이터 수', row=1, col=2)

# 그래프 표시
fig.show()

### 모델 학습 / 평가

In [6]:
# 불균형 처리 전 모델 학습
rf_original = RandomForestClassifier(random_state=42)
rf_original.fit(X_train, y_train)

# 불균형 처리 전 모델 예측 및 평가
y_pred_original = rf_original.predict(X_test)
print("불균형 처리 전 모델 성능:")
print(classification_report(y_test, y_pred_original))

print("\n" + "="*50 + "\n")

# 불균형 처리 후 모델 학습
rf_resampled = RandomForestClassifier(random_state=42)
rf_resampled.fit(X_train_resampled, y_train_resampled)

# 불균형 처리 후 모델 예측 및 평가
y_pred_resampled = rf_resampled.predict(X_test)
print("불균형 처리 후 모델 성능:")
print(classification_report(y_test, y_pred_resampled))

불균형 처리 전 모델 성능:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.77      0.86        98

    accuracy                           1.00     56962
   macro avg       0.99      0.88      0.93     56962
weighted avg       1.00      1.00      1.00     56962



불균형 처리 후 모델 성능:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.87      0.85      0.86        98

    accuracy                           1.00     56962
   macro avg       0.94      0.92      0.93     56962
weighted avg       1.00      1.00      1.00     56962

